In [1]:
# see https://medium.com/swlh/learning-with-deep-sarsa-openai-gym-c9a470d027a

import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.agents.sarsa import SARSAAgent
from rl.memory import EpisodeParameterMemory

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy
from rl.policy import EpsGreedyQPolicy
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [2]:
env = gym.make('gym_ent:ent-v1')

print(env.observation_space)
print(env.action_space)

nb_actions = env.action_space.n
obs_dim = env.observation_space.shape

Box(13, 2)
Discrete(4)


In [3]:
# Option 1 : Simple model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))
print(model.summary())

sars = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=1000, train_interval=1)
#sars = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=2000, train_interval=1)
sars.compile(optimizer='adam')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 26)                0         
_________________________________________________________________
dense (Dense)                (None, 4)                 108       
_________________________________________________________________
activation (Activation)      (None, 4)                 0         
Total params: 108
Trainable params: 108
Non-trainable params: 0
_________________________________________________________________
None


In [3]:
#     def __init__(self, nb_actions, memory, gamma=.99, batch_size=32, nb_steps_warmup=1000,
#                  train_interval=1, memory_interval=1, target_model_update=10000,
#                  delta_range=None, delta_clip=np.inf, custom_model_objects={}, **kwargs):
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))

# memory arg?
#sars = DQNAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=1000, train_interval=1)
#sars.compile(optimizer='adam')

In [3]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(64, activation='relu'))
#model.add(Activation('relu'))
model.add(Dense(32, activation='relu'))
#model.add(Activation('relu'))
model.add(Dense(16, activation='relu'))
#model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))

sars = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=1000, train_interval=50)
#sars = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=50, train_interval=1)
sars.compile(optimizer='adam')

In [4]:
sars.fit(env, nb_steps=20000, visualize=False, verbose=1)
#sars.fit(env, nb_steps=200000, visualize=False, verbose=1)
#sars.save_weights(f'sars_{ENV_NAME}_params.h5f', overwrite=True)
# 100k 8min
# 300k 25min

Training for 20000 steps ...
Interval 1 (0 steps performed)
   76/10000 [..............................] - ETA: 6s - reward: -22.7105 

/home/lkoziol/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


10000/10000 [==============================] - 10s 975us/step - reward: -8.8221
752 episodes - episode_reward: -117.325 [-200.000, 31.400] - loss: 708.941 - mean_q: 0.890

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 10s 1ms/step - reward: -25.6267
done, took 19.975 seconds


In [5]:
sars.test(env, nb_episodes=10, visualize=True)
# 8/25

Testing for 5 episodes ...
Episode 1: reward: -200.000, steps: 1
Episode 2: reward: -197.650, steps: 3
Episode 3: reward: -198.800, steps: 2
Episode 4: reward: -198.800, steps: 2
Episode 5: reward: -200.000, steps: 1


In [7]:
# memory = EpisodeParameterMemory(limit=2000, window_length=1)
# cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
#                batch_size=50, nb_steps_warmup=500, train_interval=1, elite_frac=0.05)
# cem.compile()

# cem.fit(env, nb_steps=10000, visualize=False, verbose=1)

In [8]:
#cem.test(env, nb_episodes=5, visualize=True)

In [6]:
env.render(close=True)